In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
!pip install -qU rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 53.4 MB/s eta 0:00:00


In [3]:

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
from rdkit.Chem import Lipinski, Crippen, MolSurf, GraphDescriptors, Fragments

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import inspect

# load data
train = pd.read_csv('/kaggle/input/melting-point/train.csv')
test = pd.read_csv('/kaggle/input/melting-point/test.csv')

print("="*60)
print("ULTIMATE MELTING POINT PREDICTOR")
print("Target: <10 MAE | Current: 25.40")
print("="*60)


def extract_maximum_features(smiles):
    """Extract ALL possible molecular features"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    
    features = {}
    
    # BASIC DESCRIPTORS 
    features['MolWt'] = Descriptors.MolWt(mol)
    features['MolLogP'] = Descriptors.MolLogP(mol)
    features['NumHDonors'] = Descriptors.NumHDonors(mol)
    features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
    features['TPSA'] = Descriptors.TPSA(mol)
    
    #  LIPINSKI 
    features['HeavyAtomCount'] = Lipinski.HeavyAtomCount(mol)
    features['NHOHCount'] = Lipinski.NHOHCount(mol)
    features['NOCount'] = Lipinski.NOCount(mol)
    features['NumAliphaticRings'] = Lipinski.NumAliphaticRings(mol)
    features['NumAromaticRings'] = Lipinski.NumAromaticRings(mol)
    features['NumRotatableBonds'] = Lipinski.NumRotatableBonds(mol)
    features['RingCount'] = Lipinski.RingCount(mol)
    features['NumSaturatedRings'] = Lipinski.NumSaturatedRings(mol)
    features['NumAliphaticCarbocycles'] = Lipinski.NumAliphaticCarbocycles(mol)
    features['NumAliphaticHeterocycles'] = Lipinski.NumAliphaticHeterocycles(mol)
    features['NumAromaticCarbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
    features['NumAromaticHeterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
    features['NumSaturatedCarbocycles'] = Lipinski.NumSaturatedCarbocycles(mol)
    features['NumSaturatedHeterocycles'] = Lipinski.NumSaturatedHeterocycles(mol)
    
    #  CRIPPEN 
    features['MolMR'] = Crippen.MolMR(mol)
    
    #  SURFACE AREA 
    features['LabuteASA'] = MolSurf.LabuteASA(mol)
    
    #  GRAPH DESCRIPTORS 
    try:
        features['BalabanJ'] = GraphDescriptors.BalabanJ(mol)
    except:
        features['BalabanJ'] = 0
    features['BertzCT'] = GraphDescriptors.BertzCT(mol)
    try:
        features['Ipc'] = GraphDescriptors.Ipc(mol)
    except:
        features['Ipc'] = 0
    
    #  SHAPE & CONNECTIVITY 
    features['Kappa1'] = Descriptors.Kappa1(mol)
    features['Kappa2'] = Descriptors.Kappa2(mol)
    features['Kappa3'] = Descriptors.Kappa3(mol)
    features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)
    features['Chi0v'] = Descriptors.Chi0v(mol)
    features['Chi1v'] = Descriptors.Chi1v(mol)
    features['Chi2v'] = Descriptors.Chi2v(mol)
    features['Chi3v'] = Descriptors.Chi3v(mol)
    features['Chi4v'] = Descriptors.Chi4v(mol)
    features['Chi0n'] = Descriptors.Chi0n(mol)
    features['Chi1n'] = Descriptors.Chi1n(mol)
    features['Chi2n'] = Descriptors.Chi2n(mol)
    features['Chi3n'] = Descriptors.Chi3n(mol)
    features['Chi4n'] = Descriptors.Chi4n(mol)
    
    #  ELECTRONIC 
    features['NumValenceElectrons'] = Descriptors.NumValenceElectrons(mol)
    features['NumRadicalElectrons'] = Descriptors.NumRadicalElectrons(mol)
    
    #  COUNTS 
    features['NumAtoms'] = mol.GetNumAtoms()
    features['NumHeavyAtoms'] = mol.GetNumHeavyAtoms()
    features['NumBonds'] = mol.GetNumBonds()
    features['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
    features['FormalCharge'] = Chem.GetFormalCharge(mol)
    
    #  SATURATION 
    try:
        features['FractionCsp3'] = Descriptors.FractionCSP3(mol)
    except:
        features['FractionCsp3'] = 0
    
    #  ESTATE 
    features['MaxEStateIndex'] = Descriptors.MaxEStateIndex(mol)
    features['MinEStateIndex'] = Descriptors.MinEStateIndex(mol)
    features['MaxAbsEStateIndex'] = Descriptors.MaxAbsEStateIndex(mol)
    features['MinAbsEStateIndex'] = Descriptors.MinAbsEStateIndex(mol)
    
    #  FUNCTIONAL GROUPS (EXPANDED) 
    for frag_name in ['fr_Al_COO', 'fr_Al_OH', 'fr_Ar_COO', 'fr_Ar_N', 'fr_Ar_NH',
                      'fr_Ar_OH', 'fr_COO', 'fr_C_O', 'fr_NH0', 'fr_NH1', 'fr_NH2',
                      'fr_alkyl_halide', 'fr_benzene', 'fr_ester', 'fr_ether', 
                      'fr_halogen', 'fr_ketone', 'fr_phenol', 'fr_nitro']:
        try:
            features[frag_name] = getattr(Fragments, frag_name)(mol)
        except:
            features[frag_name] = 0
    
    #  MULTIPLE FINGERPRINTS 
    # Morgan radius 1
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=1, nBits=512)
    for i, bit in enumerate(fp1):
        features[f'morgan_r1_{i}'] = int(bit)
    
    # Morgan radius 2
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
    for i, bit in enumerate(fp2):
        features[f'morgan_r2_{i}'] = int(bit)
    
    # Morgan radius 3
    fp3 = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=512)
    for i, bit in enumerate(fp3):
        features[f'morgan_r3_{i}'] = int(bit)
    
    return features

print("\nExtracting MAXIMUM molecular features...")

train_mol = train['SMILES'].apply(extract_maximum_features)
train_mol_df = pd.DataFrame(list(train_mol)).fillna(0)

test_mol = test['SMILES'].apply(extract_maximum_features)
test_mol_df = pd.DataFrame(list(test_mol)).fillna(0)

print(f"✓ Extracted {train_mol_df.shape[1]} molecular features")


# Group features
id_cols = ['id', 'SMILES', 'Tm']
feature_cols = [col for col in train.columns if col not in id_cols]
X_group = train[feature_cols]
X_test_group = test[feature_cols]

# Removing zero variance from Group
zero_var_group = X_group.columns[X_group.std() == 0]
X_group = X_group.drop(columns=zero_var_group)
X_test_group = X_test_group.drop(columns=zero_var_group)

# Combine
X_combined = pd.concat([X_group, train_mol_df], axis=1)
X_test_combined = pd.concat([X_test_group, test_mol_df], axis=1)

# Removin zero variance from combined
zero_var = X_combined.columns[X_combined.std() == 0]
X_combined = X_combined.drop(columns=zero_var)
X_test_combined = X_test_combined.drop(columns=zero_var)

y = train['Tm']

print(f"\nFinal features: {X_combined.shape[1]}")


class RegressionTrainer:
    def __init__(self, model, n_folds=5, random_state=42):
        self.model = model
        self.n_folds = n_folds
        self.random_state = random_state
        self.kfold = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        
    def train_predict(self, X_train, y_train, X_test, verbose=True):
        oof_preds = np.zeros(len(X_train))
        test_preds = np.zeros(len(X_test))
        
        for fold, (train_idx, val_idx) in enumerate(self.kfold.split(X_train), 1):
            if verbose:
                print(f"Fold {fold}/{self.n_folds}...", end=" ")
            
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            fold_model = clone(self.model)
            
            if hasattr(fold_model, 'fit') and 'eval_set' in inspect.signature(fold_model.fit).parameters:
                fit_params = {'eval_set': [(X_val, y_val)]}
                if 'verbose' in inspect.signature(fold_model.fit).parameters:
                    fit_params['verbose'] = False
                fold_model.fit(X_tr, y_tr, **fit_params)
            else:
                fold_model.fit(X_tr, y_tr)
            
            oof_preds[val_idx] = fold_model.predict(X_val)
            test_preds += fold_model.predict(X_test) / self.n_folds
            
            fold_mae = mean_absolute_error(y_val, oof_preds[val_idx])
            if verbose:
                print(f"MAE: {fold_mae:.4f}")
        
        overall_mae = mean_absolute_error(y_train, oof_preds)
        if verbose:
            print(f"Overall: {overall_mae:.4f}\n")
        
        return oof_preds, test_preds, overall_mae


print("\n" + "="*60)
print("TRAINING ENSEMBLE OF MODELS")
print("="*60)

all_oof = []
all_test = []
all_names = []

# Model 1: XGBoost v1
print("\n1. XGBoost V1 (Deep)")
xgb1 = XGBRegressor(n_estimators=5000, max_depth=10, learning_rate=0.005, 
                    subsample=0.8, colsample_bytree=0.7, min_child_weight=2,
                    gamma=0.2, reg_alpha=1, reg_lambda=2, random_state=42)
trainer = RegressionTrainer(xgb1)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'XGB1({mae:.2f})')

# Model 2: XGBoost v2
print("2. XGBoost V2 (Wide)")
xgb2 = XGBRegressor(n_estimators=4000, max_depth=6, learning_rate=0.008,
                    subsample=0.85, colsample_bytree=0.85, min_child_weight=5,
                    gamma=0.1, reg_alpha=0.5, reg_lambda=1, random_state=123)
trainer = RegressionTrainer(xgb2)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'XGB2({mae:.2f})')

# Model 3: LightGBM v1
print("3. LightGBM V1")
lgb1 = LGBMRegressor(n_estimators=5000, num_leaves=100, learning_rate=0.005,
                     subsample=0.8, colsample_bytree=0.7, min_child_samples=10,
                     reg_alpha=1, reg_lambda=2, random_state=42, verbosity=-1, force_row_wise=True)
trainer = RegressionTrainer(lgb1)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'LGB1({mae:.2f})')

# Model 4: LightGBM v2
print("4. LightGBM V2")
lgb2 = LGBMRegressor(n_estimators=4000, num_leaves=50, learning_rate=0.008,
                     subsample=0.85, colsample_bytree=0.85, min_child_samples=20,
                     reg_alpha=0.5, reg_lambda=1, random_state=123, verbosity=-1, force_row_wise=True)
trainer = RegressionTrainer(lgb2)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'LGB2({mae:.2f})')

# Model 5: CatBoost v1
print("5. CatBoost V1")
cat1 = CatBoostRegressor(iterations=5000, depth=10, learning_rate=0.005,
                         l2_leaf_reg=5, random_state=42, verbose=False)
trainer = RegressionTrainer(cat1)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'CAT1({mae:.2f})')

# Model 6: CatBoost v2
print("6. CatBoost V2")
cat2 = CatBoostRegressor(iterations=4000, depth=6, learning_rate=0.008,
                         l2_leaf_reg=3, random_state=123, verbose=False)
trainer = RegressionTrainer(cat2)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'CAT2({mae:.2f})')

# Model 7: Extra Trees
print("7. Extra Trees")
et = ExtraTreesRegressor(n_estimators=500, max_depth=20, min_samples_split=5,
                         min_samples_leaf=2, random_state=42, n_jobs=-1)
trainer = RegressionTrainer(et)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'ET({mae:.2f})')

# Model 8: Random Forest
print("8. Random Forest")
rf = RandomForestRegressor(n_estimators=500, max_depth=20, min_samples_split=5,
                           min_samples_leaf=2, random_state=42, n_jobs=-1)
trainer = RegressionTrainer(rf)
oof, test_pred, mae = trainer.train_predict(X_combined, y, X_test_combined)
all_oof.append(oof)
all_test.append(test_pred)
all_names.append(f'RF({mae:.2f})')


print("\n" + "="*60)
print("ADVANCED STACKING")
print("="*60)

# Stack all predictions
oof_stack = np.column_stack(all_oof)
test_stack = np.column_stack(all_test)

print(f"\nStacking {len(all_names)} models:")
for name in all_names:
    print(f"  - {name}")

#  multiple meta-learners
meta_models = {
    'Ridge': Ridge(alpha=10, random_state=42),
    'Lasso': Lasso(alpha=0.5, random_state=42),
    'ElasticNet': ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, num_leaves=31, learning_rate=0.1, random_state=42, verbosity=-1),
}

best_mae = float('inf')
best_model = None
best_name = None

print("\nTesting meta-learners:")
for name, meta in meta_models.items():
    meta.fit(oof_stack, y)
    meta_oof = meta.predict(oof_stack)
    mae = mean_absolute_error(y, meta_oof)
    print(f"  {name:15s}: MAE = {mae:.4f}")
    
    if mae < best_mae:
        best_mae = mae
        best_model = meta
        best_name = name

print(f"\n Best meta-learner: {best_name} (MAE: {best_mae:.4f})")

# Final predictions
final_test = best_model.predict(test_stack)


submission = pd.DataFrame({
    'id': test['id'],
    'Tm': final_test
})

submission.to_csv('submission.csv', index=False)

print(f"\nPrediction range: {final_test.min():.1f}K to {final_test.max():.1f}K")
print(f"Training range: {y.min():.1f}K to {y.max():.1f}K")
print("\n submission.csv created!")

ULTIMATE MELTING POINT PREDICTOR
Target: <10 MAE | Current: 25.40

Extracting MAXIMUM molecular features...


[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerator
[21:41:49] DEPRECATION WARNING: please use MorganGenerat

✓ Extracted 2117 molecular features

Final features: 2411

TRAINING ENSEMBLE OF MODELS

1. XGBoost V1 (Deep)
Fold 1/5... MAE: 29.2181
Fold 2/5... MAE: 28.6370
Fold 3/5... MAE: 28.4415
Fold 4/5... MAE: 28.9728
Fold 5/5... MAE: 27.9469
Overall: 28.6435

2. XGBoost V2 (Wide)
Fold 1/5... MAE: 29.1289
Fold 2/5... MAE: 28.6755
Fold 3/5... MAE: 28.5664
Fold 4/5... MAE: 28.9505
Fold 5/5... MAE: 27.8408
Overall: 28.6326

3. LightGBM V1
Fold 1/5... MAE: 29.8898
Fold 2/5... MAE: 28.8019
Fold 3/5... MAE: 29.0818
Fold 4/5... MAE: 29.7855
Fold 5/5... MAE: 29.2899
Overall: 29.3698

4. LightGBM V2
Fold 1/5... MAE: 30.0605
Fold 2/5... MAE: 30.0088
Fold 3/5... MAE: 29.9023
Fold 4/5... MAE: 29.9823
Fold 5/5... MAE: 30.2043
Overall: 30.0316

5. CatBoost V1
Fold 1/5... MAE: 30.1054
Fold 2/5... MAE: 29.6665
Fold 3/5... MAE: 29.0574
Fold 4/5... MAE: 29.8580
Fold 5/5... MAE: 28.6356
Overall: 29.4649

6. CatBoost V2
Fold 1/5... MAE: 30.0038
Fold 2/5... MAE: 29.9668
Fold 3/5... MAE: 29.8269
Fold 4/5... MAE: 29.